In [1]:
from custom.pipeline import ClassicPipeline, TransformerPipeline, txt2list, PreProcess
import spacy

In [5]:
preprocess = PreProcess()
batch_sequence = preprocess.load_text(
    txt_path='/Users/sergicastellasape/Repos/zeta-alpha/datasets/quora_questions.txt')
batch_tensors, indices_chunk = preprocess.forward(batch_sequence)

In [7]:
i = 0
print(batch_sequence[i])
print(indices_chunk[i])
print(batch_tensors.size())

Are convolutional neural networks useful for tasks other than image classification?
[(0,), (1,), (2, 3, 4, 5, 6, 7, 8), (9,), (10,), (11,), (12,), (13,), (14, 15), (16,), (17,)]
torch.Size([301, 57, 768])


In [8]:
def expand_indices(indices_batch, target_lengths):

    assert len(indices_batch) == len(target_lengths)

    for b, target_length in enumerate(target_lengths):
        if target_length > len(indices_batch[b]):

            while indices_batch[b][0][0] > 0:
                first_idx = indices_batch[b][0][0]
                indices_batch[b].insert(0, (first_idx-1,))

            while indices_batch[b][-1][-1] < target_length - 1:
                last_idx = indices_batch[b][-1][-1]
                indices_batch[b].append((last_idx+1,))
    
    return indices_batch


a = [[(1,), (2,), (3, 4)], [(0,), (1,), (2, 3), (4,)]]
target = [6, 10]

I = expand_indices(a, target)
print(I)

im in initial
[[(0,), (1,), (2,), (3, 4), (5,)], [(0,), (1,), (2, 3), (4,), (5,), (6,), (7,), (8,), (9,)]]


In [1]:
print(batch_sequence[0])
print(batch_tensors[0])
indices_chunk[0]

NameError: name 'batch_sequence' is not defined

In [6]:
"""pp = PreProcess()
cm = CompressionModel()
for doc_path in train_dir:
    batch_sequence = pp.load_text(txt_path=doc_path)
    batch_tensors, idx_chunk = pp.forward(batch_sequence)
    compact_rep = cm.forward(batch_tensors, idx_chunk)
    loss.backward()"""

'pp = PreProcess()\ncm = CompressionModel()\nfor doc_path in train_dir:\n    batch_sequence = pp.load_text(txt_path=doc_path)\n    batch_tensors, idx_chunk = pp.forward(batch_sequence)\n    compact_rep = cm.forward(batch_tensors, idx_chunk)\n    loss.backward()'

In [2]:
nlp = spacy.load('en_trf_bertbaseuncased_lg')
classic_nlp = spacy.load('en_core_web_sm')

In [3]:
#text = "What is a bayesian neural network?"
cls_pipeline = ClassicPipeline()
trf_pipeline = TransformerPipeline()

In [9]:
texts = txt2list(
    txt_path='/Users/sergicastellasape/Repos/zeta-alpha/datasets/quora_questions.txt')
docs = trf_pipeline.make_docs(texts[0:10])
cls_docs = cls_pipeline.make_docs(texts)

In [5]:
#texts = ['What are Convolutional Neural Networks this is.', 'Are bayesian networks new.']

spacy_tokenization = cls_pipeline.make_base_tokenization(cls_docs)
base_chunks = cls_pipeline.make_noun_phrase_chunks(cls_docs)

brackets, chunk2spacy_idx = cls_pipeline.compact_tokens(texts)
spacy2trf_idx = trf_pipeline.spacy2trf_indices(docs)

chunk2trf_idx = trf_pipeline.chunk2trf_indices(chunk2spacy_idx, spacy2trf_idx)

In [8]:
i = 3

print("Spacy chunks: ", base_chunks[i])
print("Spacy tokens: ", spacy_tokenization[i])
print("Transformer wordpieces: ", docs[i]._.trf_word_pieces_)
print("Chunk to trans idx: ", chunk2trf_idx[i])

Spacy chunks:  [('what',), ('a', 'dirichlet', 'process'), ('layman', "'s", 'terms')]
Spacy tokens:  ['bayesian', 'inference', ':', 'what', 'is', 'a', 'dirichlet', 'process', 'in', 'layman', "'s", 'terms', '?']
Transformer wordpieces:  ['[CLS]', 'bay', '##esian', 'inference', ':', 'what', 'is', 'a', 'dir', '##ich', '##let', 'process', 'in', 'lay', '##man', "'", 's', 'terms', '?', '[SEP]']
Chunk to trans idx:  [(1, 2), (3,), (4,), (5,), (6,), (7, 8, 9, 10, 11), (12,), (13, 14, 15, 16, 17), (18,)]


In [7]:
classic_doc = classic_nlp(text)
chunks_tokenization = [chunk.text for chunk in classic_doc.noun_chunks]
spacy_tokenization = [word.text for word in doc]

NameError: name 'text' is not defined

In [ ]:
print(chunks_tokenization)
print(spacy_tokenization)
spacy

In [ ]:
from spacy.gold import align
a = ['What', 'is', 'a bayesian neural network', '?']
b = ['What', 'is', 'a', 'bayesian', 'neural', 'network', '?']
cost, a2b, b2a, a2b_multi, b2a_multi = align(a, b)
print("Misaligned tokens:", cost)  # 2
print("One-to-one mappings a -> b", a2b)  # array([0, 1, 2, 3, -1, -1, 5, 6])
print("One-to-one mappings b -> a", b2a)  # array([0, 1, 2, 3, 5, 6, 7])
print("Many-to-one mappings a -> b", a2b_multi)  # {4: 4, 5: 4}
print("Many-to-one mappings b-> a", b2a_multi)  # {}

In [ ]:
a = [(1,), (2,), (3, 4,), (5,)]
print(a[1:])

In [ ]:
s = 'sergi     castella'
s.split()

In [ ]:
print(s)

In [11]:
import torch
t = torch.Tensor([1, 2, 3, 4])

In [14]:
t.remove(2)

AttributeError: 'Tensor' object has no attribute 'remove'